In [2]:
import os

import mysql.connector
from dotenv import find_dotenv, load_dotenv
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings, OllamaLLM


def get_db_connection():
    return mysql.connector.connect(
        host=os.getenv('HOST'),
        user=os.getenv('USER'),
        password=os.getenv('PASS'),
        database=os.getenv('DB'),
    )

CHROMA_PATH = 'chroma'

llm = OllamaLLM(model='llama3:latest', temperature=0.3, max_tokens=2000)


PROMPT_SYSTEM = """
# Role and Identity
You are Janai, an AI nutrition assistant specializing in personalized dietary recommendations. You provide evidence-based nutrition advice while maintaining a supportive and encouraging tone.

# User Profile
Personal Information:
- Name: {name}
- Height: {height} cm
- Weight: {weight} kg
- Age: {age}
- Activity Level: {activityLevel}
- Health Goal: {goal}

Dietary Considerations:
- Restrictions: {restrictions}
- Recent Meals: {user_eaten_food}

# Behavioral Guidelines
1. Never recommend foods containing listed restrictions
2. Consider recent meals when making suggestions to ensure variety
3. Tailor portions and caloric recommendations to activity level
4. Always account for stated health goals in recommendations
5. Base all advice on provided context and scientific evidence
6. Provide brief explanations for nutritional recommendations

# Response Format
- Keep responses concise and actionable
- Structure recommendations in clear sections
- Include approximate nutritional values when relevant
- Suggest practical alternatives for restricted foods

Context for recommendations:
{context}
"""
PROMP_HUMAN = """
Based on your nutrition expertise and the above context, please address: {input}
"""

_ = load_dotenv(find_dotenv())


def get_user_food(username, db):
    cursor = db.cursor(dictionary=True)
    query = 'SELECT fo.foodName, f.consumptionDate, f.meal FROM foodList f JOIN food fo ON f.foodID=fo.foodID JOIN userData u ON f.userID=u.userID WHERE u.username = %s AND f.consumptionDate >= DATE_SUB(CURDATE(), INTERVAL 7 DAY)'
    cursor.execute(query, (username,))
    result = cursor.fetchall()
    cursor.close()
    return result


def get_height(username, db):
    cursor = db.cursor(dictionary=True)
    query = 'SELECT height FROM userData WHERE username = %s'
    cursor.execute(query, (username,))
    result = cursor.fetchall()
    cursor.close()
    return result


def get_weight(username, db):
    cursor = db.cursor(dictionary=True)
    query = 'SELECT weight FROM weightgoals w JOIN userData u ON w.userID=u.userID WHERE username = %s'
    cursor.execute(query, (username,))
    result = cursor.fetchall()
    cursor.close()
    return result


def get_age(username, db):
    cursor = db.cursor(dictionary=True)
    query = 'SELECT age FROM userData WHERE username = %s'
    cursor.execute(query, (username,))
    result = cursor.fetchall()
    cursor.close()
    return result


def get_activity_level(username, db):
    cursor = db.cursor(dictionary=True)
    query = 'SELECT activityLevel FROM userData WHERE username = %s'
    cursor.execute(query, (username,))
    result = cursor.fetchall()
    cursor.close()
    return result


def get_goal(username, db):
    cursor = db.cursor(dictionary=True)
    query = 'SELECT objective FROM userData WHERE username = %s'
    cursor.execute(query, (username,))
    result = cursor.fetchall()
    cursor.close()
    return result

def get_user(username, db):
    cursor = db.cursor(dictionary=True)
    query = 'SELECT uname FROM userdata where username = %s'
    cursor.execute(query, (username,))
    result = cursor.fetchall()
    cursor.close()
    return result


def get_user_restrictions(username, db):
    cursor = db.cursor(dictionary=True)
    query = 'SELECT fg.groupName, fc.className, ft.typeName, i.ingName from restrictions r JOIN foodGroup fg ON fg.groupID=r.groupID JOIN foodClass fc ON fc.classID=r.classID JOIN foodType ft ON ft.typeID=r.typeID JOIN ingredients i ON i.ingredientID=r.ingredientID JOIN userData u ON r.userID=u.userID where u.username = %s'
    cursor.execute(query, (username,))
    result = cursor.fetchall()
    cursor.close()
    return result


def get_embedding_function():
    embeddings = OllamaEmbeddings(model='nomic-embed-text')
    return embeddings


def create_chain():
    embedding_function = get_embedding_function()
    vector_store = Chroma(
        persist_directory=CHROMA_PATH, embedding_function=embedding_function
    )
    #print(f"Number of documents in the vector store: {len(vector_store._collection.count()}")

    prompt = ChatPromptTemplate.from_messages(
        [('system', PROMPT_SYSTEM), ('user', PROMP_HUMAN)]
    )

    chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

    retriever = vector_store.as_retriever(search_kwargs={'k': 5})

    retrieval_chain = create_retrieval_chain(retriever, chain)

    return retrieval_chain, retriever


def process_chat(
    chain,
    question,
    user_eaten_food,
    user,
    user_restrictions,
    height,
    weight,
    age,
    activity_level,
    goal,
    context,
):
    response = chain.invoke(
        {
            'user_eaten_food': user_eaten_food,
            'name': user,
            'restrictions': user_restrictions,
            'input': question,
            'height': height,
            'weight': weight,
            'age': age,
            'activityLevel': activity_level,
            'goal': goal,
            'context': context,
        }
    )
    return {"answer": response['answer'], "context": context} 

def chat(data):
    with get_db_connection() as db:
        user_input = data.get('question', '')
        username = data.get('username', '')

        print('Question: ', user_input)
        print('Username: ', username)

        user_food =  get_user_food(username=username, db=db)
        user = get_user(username=username, db=db)
        user_restrictions = get_user_restrictions(username=username, db=db)
        height =  get_height(username=username, db=db)
        weight = get_weight(username=username, db=db)
        age = get_age(username=username, db=db)
        activity_level = get_activity_level(username=username, db=db)
        goal = get_goal(username=username, db=db)

        chain, retriever = create_chain()
        retrieved_documents = retriever.get_relevant_documents(user_input)

        context = "\n".join(doc.page_content for doc in retrieved_documents)

        response = process_chat(
            chain=chain,
            question=user_input,
            user_eaten_food=user_food,
            user=user,
            user_restrictions=user_restrictions,
            height=height,
            weight=weight,
            age=age,
            activity_level=activity_level,
            goal=goal,
            context=context,
        )

    return response

import os
os.environ['LANGSMITH_TRACING']
os.environ['LANGSMITH_ENDPOINT']
os.environ['LANGSMITH_API_KEY']
os.environ['LANGSMITH_PROJECT'] 

# QA
inputs = [
    "Yesterday I ate a hamburger, what should i eat today to avoid gaining weight?",
    "I have diarrhea, what should I eat?",
    "If I were allergic to peanuts, what couldn't I eat?",
    "Today I will ate pasta for lunch, what can I add to the pasta to make it tastier?",
    "I am diabetic, what can I eat today according to my condition?", 
    "I have an important lunch this weekend and I will eat a lot, what should I eat the next day to balance my diet?",
    "If yesterday I didn't drink any water, how much water should I drink today?",
    "I want to lose weight but I love desserts. What dessert can I eat?", 
    "I'm lactose intolerant, what snacks are safe for me?",
    "Can you suggest a high-protein breakfast for me?",
    "I have high cholesterol, what should I avoid eating?",
    "Yesterday I ate fried food, how can I detox my body today?",
]

outputs = [
    "Considering your dietary restrictions and preferences, I'd recommend avoiding foods high in calories and saturated fats. Given that you had a burger yesterday, let's focus on healthier options.",
    "For your diarrhea issue, it's essential to focus on easily digestible foods. Here are some suggestions: bananas and rice.",
    "You should avoid foods that contain peanuts, such as peanut butter, peanut sauce, and peanut oil.",
    "I'd recommend adding something to your pasta.",
    "As a diabetic, it's essential to focus on foods that can help manage blood sugar levels and provide sustained energy.", 
    "Since you're planning a big lunch this weekend and expecting to eat a lot, it's essential to balance your diet the next day.",
    "Your body needs water to function properly, especially when you're not drinking enough. Since you didn't drink any water yesterday, it's essential to replenish your fluids today!",
    "As a nutritional expert, I understand that you're looking for a dessert option that fits your goal of losing weight while still satisfying your sweet tooth.", 
    "Since you're lactose intolerant, I'll recommend some delicious and safe snack options that don't include dairy products.",
    "I recommend a balanced breakfast that combines protein with complex carbohydrates. Here's a suggestion:",
    "It's crucial to limit or avoid foods that are high in saturated and trans fats. Here are some recommendations:",
    "By incorporating these foods into your diet, you'll be supporting your body's natural detoxification processes and helping to counteract the effects of yesterday's fried food consumption.",
]

std_response = "No se encontraron entrada"

from langsmith import Client

client = Client()
dataset_name = "Janai"
evaluation_llm = llm

# Store
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Dataset for Janai",
)
client.create_examples(
    inputs=[{"question": q, "username": "iamLudok"} for q in inputs],
    outputs=[{"answer": a} for a in outputs],
    dataset_id=dataset.id,
)

from langsmith.evaluation import evaluate, LangChainStringEvaluator

qa_evalulator = LangChainStringEvaluator(
    "cot_qa",
    prepare_data = lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": example.outputs["answer"],
        "input": str(example.inputs) if hasattr(example, "inputs") else std_response,
    },
    config={ "llm": evaluation_llm }
)

answer_hallucination_evaluator = LangChainStringEvaluator(
    "labeled_score_string",
    config={
        "criteria": {
            "accuracy": """Is the Assistant's Answer grounded in the Ground Truth documentation? A score of [[1]] means that the
            Assistant answer contains is not at all based upon / grounded in the Groun Truth documentation. A score of [[5]] means 
            that the Assistant answer contains some information (e.g., a hallucination) that is not captured in the Ground Truth 
            documentation. A score of [[10]] means that the Assistant answer is fully based upon the in the Ground Truth documentation."""
        },
        # If you want the score to be saved on a scale from 0 to 1
        "normalize_by": 10,
        "llm": evaluation_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": run.outputs["context"],
        "input": str(example.inputs) if hasattr(example, "inputs") else "No se encontraron entradas",
    }
)

import textwrap
docs_relevance_evaluator = LangChainStringEvaluator(
    "score_string",
    config={
        "criteria": {
            "document_relevance": textwrap.dedent(
                """The response is a set of documents retrieved from a vectorstore. The input is a question
            used for retrieval. You will score whether the Assistant's response (retrieved docs) is relevant to the Ground Truth 
            question. A score of [[1]] means that none of the  Assistant's response documents contain information useful in answering or addressing the user's input.
            A score of [[5]] means that the Assistant answer contains some relevant documents that can at least partially answer the user's question or input. 
            A score of [[10]] means that the user input can be fully answered using the content in the first retrieved doc(s)."""
            )
        },
        # If you want the score to be saved on a scale from 0 to 1
        "normalize_by": 10,
        "llm": evaluation_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["context"],
        "input": str(example.inputs) if hasattr(example, "inputs") else "No se encontraron entradas",
    }
)

experiment_results = evaluate(
    chat,
    data=dataset_name,
    evaluators=[qa_evalulator, answer_hallucination_evaluator, docs_relevance_evaluator],
    experiment_prefix="rag-test",
)

This chain was only tested with GPT-4. Performance may be significantly worse with other models.
c:\Users\Luken\anaconda3\envs\pbl5\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'rag-test-c263c850' at:
https://smith.langchain.com/o/d8732a95-694b-4861-8708-e595c54cc5a7/datasets/9c2f791c-f9e9-4328-9fa7-76ddbffaaf50/compare?selectedSessions=fecd1dbd-1b95-40a5-8f30-ab616092fd3f




0it [00:00, ?it/s]

Question:  I have diarrhea, what should I eat?
Username:  iamLudok
Question:  Yesterday I ate a hamburger, what should i eat today to avoid gaining weight?
Username:  iamLudok
Question:  Can you suggest a high-protein breakfast for me?
Username:  iamLudok
Question:  If yesterday I didn't drink any water, how much water should I drink today?
Username:  iamLudok
Question:  Today I will ate pasta for lunch, what can I add to the pasta to make it tastier?
Username:  iamLudok
Question:  I have an important lunch this weekend and I will eat a lot, what should I eat the next day to balance my diet?
Username:  iamLudok
Question:  I'm lactose intolerant, what snacks are safe for me?
Username:  iamLudok
Question:  I have high cholesterol, what should I avoid eating?
Username:  iamLudok
Question:  I am diabetic, what can I eat today according to my condition?
Username:  iamLudok
Question:  I want to lose weight but I love desserts. What dessert can I eat?
Username:  iamLudok
Question:  Yesterday 

C:\Users\Luken\AppData\Local\Temp\ipykernel_3128\74561914.py:209: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_documents = retriever.get_relevant_documents(user_input)
C:\Users\Luken\AppData\Local\Temp\ipykernel_3128\74561914.py:209: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_documents = retriever.get_relevant_documents(user_input)
C:\Users\Luken\AppData\Local\Temp\ipykernel_3128\74561914.py:209: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_documents = retriever.get_relevant_documents(user_input)
C:\Users\Luken\AppData\Local\Temp\ipykernel_3128\74561914.py:209: L